<a href="https://colab.research.google.com/github/duongquangvinh/Fundamental-Machine-Learning-model/blob/main/catdog_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import pandas as pd
import os
import torch
import torch.optim as optim
import torch.nn as nn
import cv2
import torchvision
import torch.nn.functional as F
from torchvision import transforms
import copy
import tqdm
from PIL import Image
import random

In [41]:
transform_train = transforms.Compose([transforms.Resize((224,224)),
                                transforms.RandomHorizontalFlip(0.5),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_test = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

## Train Test Split

In [ ]:
!unzip drive/MyDrive/Dataset/dog_cat.zip

In [ ]:
data_dir = "./dog_cat"
dog = []
cat = []
for img in os.listdir(data_dir):
    label = img.split(".")[0]
    img_path = os.path.join(data_dir, img)
    if label == "dog":
        dog.append(img_path)
    else:
        cat.append(img_path)

rate = 0.2
test_cat = random.sample(cat, int(rate*len(cat)))
test_dog = random.sample(dog, int(rate*len(dog)))

train_cat = list(set(cat) ^ set(test_cat))
train_dog = list(set(dog) ^ set(test_dog))

trainfile = train_cat + train_dog
testfile = test_cat + test_dog
print(testfile)

## Data Preprocess

In [6]:
class CatDogDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform = None):
        self.data_dir = data_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.data_dir)
    
    def __getitem__(self, idx):
        img_path = self.data_dir[idx]
        img = Image.open(img_path)
        img_transform = self.transform(img)
        label = img_path.split("/")[-1].split(".")[0]
        if label == "dog":
            label = 1
        elif label == "cat":
            label = 0
        return img_transform, label

In [42]:
trainset = CatDogDataset(trainfile, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2, drop_last=True)

testset = CatDogDataset(testfile, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2, drop_last=True)

In [43]:
testset[0][0].shape

torch.Size([3, 224, 224])

In [45]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2, padding=0)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=0)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=0)
        self.bn3 = nn.BatchNorm2d(64)

        self.fc1 = nn.Linear(3*3*64, 20)
        self.fc2 = nn.Linear(20, 2)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        # x = x.view(x.size(0), -1)
        x = x.view(-1, 3*3*64)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [46]:
net = LeNet()
net = net.to("cuda")

In [49]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr = 0.001)

In [51]:
best_acc = 0
for epoch in range(3):
    net.train()
    running_loss=0
    for batch_idx, (img,label) in enumerate(trainloader):
        img = img.to("cuda")
        label = label.to("cuda")
        optimizer.zero_grad()
        output = net(img)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print("training loss:", running_loss/len(trainloader))

    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for _, (img,label) in enumerate(testloader):
            img = img.to("cuda")
            label = label.to("cuda")

            output = net(img)
            loss = criterion(output, label)
            test_loss += loss.item()
            _, predict = torch.max(output.data, 1)
            total += label.size(0)
            correct += (predict == label).sum().item()
    print(f"Test loss: {test_loss/len(testloader)}")
    print(f"Accuracy: {100*correct/total}")

    acc = 100*correct/total
    if acc > best_acc:
        print("Saving...")
        state = {
            "net": net.state_dict(),
            "acc": acc,
            "epoch": epoch
        }
        os.makedirs("./checkpoint/", exist_ok=True)
        torch.save(state, "./checkpoint/dogcat.pth")
        best_acc = acc

training loss: 0.4827803533554077
Test loss: 0.4724229831153002
Accuracy: 78.22516025641026
Saving...
training loss: 0.4501970765113831
Test loss: 0.4170389840236077
Accuracy: 81.61057692307692
Saving...
training loss: 0.426973827791214
Test loss: 0.3931693587547693
Accuracy: 82.51201923076923
Saving...
